# Getting Start

### Installation of JanusQ
JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended, as all functions have been tested on it.
Linux (Ubuntu 22.04 latest) platform and Python (3.10) is prefered when instaling by wheel or soruce code.

**From docker.**
Pull docker using docker 
        
        docker pull janusq/janusq:latest

The code is in "/JanusQ". The examples that can be directly run is in "/JanusQ/examples"

**From wheel.**
Download janusq.whl from "JanusQ-main/dist".

        pip install janusq.whl

**From source code.**
Run following commends.

        git clone git@github.com:JanusQ/JanusQ.git
        cd JanusQ
        pip install -r requirements.txt

### Structure of JanusQ

- JanusCT
  - vectorization
    - janusq/analysis/vectorization.py: This python script holds code related to the generation of path table and the vecterization of circuits
  - fidelity preidiction
    - janusq/analysis/fidelity_prediction.py: This python script holds code related to training a model and using it to predict the fidelity of a circuit.
  - fidelity optimization
    -  janusq/optimizations/mapping/mapping_ct.py：This python script holds code related to a typical compilation flow includes routing and scheduling. The routing pass transforms the circuit to satisfy the processor topology.
  - unitary decomposition
    - janusq/analysis/unitary_decompostion.py: This python script holds code related to takes a unitary as input and decomposes it into matrices of basic gates, resulting in an equivalent circuit.
  - bug identification: identify the potential bugs in the quantum algorithm implementation.
- JanusFEM
  - benchmarking.py. Generate a circuit for measuring calibration matrices.
  - mitigation.py. Implement iterative calibration, which includes quantifying interactions between qubits and constructing Bayesian networks.
  - tools.py. Implement data format conversion.
- HyQSAT
  - hyqsat/common: This dir stores common functions about cnf files, such as readCNF.
  - hyqsat/solveSatBy**.py: The main python function; use to solve sat problem.
- time crystal

### Note
1. Permission denied 'minisat_core'  
This file have no executable permission. You need run `chmod +x janusq/hyqsat/minisat_core` in bash.

### Test JanusQ

In [1]:
import os
os.chdir("..")
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

#### Test QuCT

In [2]:
# test quct
from janusq.data_objects.backend import LinearBackend
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel

# define a 10-qubit backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

up_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
up_model.train(circuits, multi_process=False, remove_redundancy = False)

100%|██████████| 300/300 [00:00<00:00, 389.73it/s]


#### Test QuFEM

In [3]:
# test readout calibration
from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.data_objects.backend import LinearBackend
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.optimizations.readout_mitigation.fem import  EnumeratedProtocol
from janusq.data_objects.algorithms import get_algorithm_circuits

n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algorithm_circuits(n_qubits, backend, algs = ['ghz'])[0]

samples = 10000

simulator = NoisySimulator(
    backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)


protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()

all_statuscnts = [
    simulator.execute(cir_protocol, samples)
    for cir_protocol in circuits_protocol
]

print(all_statuscnts)


/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/chutianyao/anaconda3/envs/janusq/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


[{'011': 10, '110': 9, '001': 634, '100': 335, '101': 28, '010': 279, '000': 8705}, {'111': 12, '011': 290, '000': 831, '010': 24, '101': 348, '100': 42, '001': 8453}, {'111': 3, '011': 23, '110': 13, '001': 609, '100': 373, '101': 21, '000': 8680, '010': 278}, {'101': 1, '011': 620, '111': 32, '010': 8863, '000': 97, '100': 2, '001': 6, '110': 379}, {'101': 2, '110': 30, '001': 83, '011': 8664, '111': 365, '000': 14, '010': 842}, {'011': 656, '111': 23, '100': 5, '001': 6, '110': 370, '000': 89, '010': 8851}, {'011': 20, '110': 12, '100': 344, '001': 555, '101': 20, '010': 323, '000': 8726}, {'011': 264, '111': 12, '010': 23, '000': 796, '101': 349, '110': 1, '100': 32, '001': 8523}, {'101': 27, '110': 7, '100': 363, '001': 646, '111': 1, '011': 26, '010': 273, '000': 8657}, {'011': 1, '111': 15, '000': 274, '010': 6, '001': 19, '100': 8803, '110': 300, '101': 582}, {'011': 7, '111': 309, '110': 27, '001': 262, '100': 814, '000': 21, '101': 8560}, {'011': 1, '111': 19, '100': 8782, '1

#### Test HyQSAT

In [4]:
# test hyqsat calibration
from janusq.hyqsat import readCNF

# input cnf flie
file_path = "cnf_examples/UF50/uf50-01.cnf"
print(readCNF(file_path))

([(-3, 7, 36), (-3, -42, -48), (-41, -47, -49), (8, 17, -40), (-21, -31, -39), (-22, 36, 49), (14, 27, 38), (6, 15, -18), (6, 7, -43), (-7, 23, 34), (2, -13, 14), (2, -42, 47), (3, -33, -35), (40, 44, 49), (31, 36, 50), (-3, -36, -37), (26, -29, 43), (15, 29, -45), (-11, 18, 24), (6, -26, -47), (-10, -33, -50), (6, 16, 32), (-34, 37, 41), (7, -17, -28), (19, -44, 46), (7, 22, -48), (3, 34, 39), (31, -43, 46), (23, -27, 32), (-18, 37, -50), (5, 11, 20), (6, -24, -45), (-14, -23, -34), (20, 21, -22), (-17, 24, 50), (-24, -25, -27), (3, 21, 35), (-26, -36, 47), (-28, -45, 49), (-6, 12, -21), (-15, -17, -39), (2, -14, 41), (-23, 25, 36), (-3, -39, -40), (20, 35, 50), (27, 31, -39), (-15, -40, 45), (34, 35, 50), (-1, 12, -48), (18, -30, -35), (-24, -25, 27), (-4, -12, -33), (-24, -37, -43), (31, -37, -44), (-9, 14, -38), (-16, 33, 34), (4, -5, -35), (-3, -19, -21), (-29, -35, -36), (7, 36, -43), (14, 30, 41), (-7, -24, -35), (6, 35, -42), (-1, -15, 39), (-16, 27, 49), (-10, -37, 49), (-3, -